https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html

https://github.com/pytorch/examples/blob/main/word_language_model/model.py

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import numpy as np
import random
import pandas as pd

In [3]:
from transformer import TransformerModel

In [14]:
from net import Net_CBOW
import numpy as np
version = "april27_WT2_nodatalim_10epoch_128dim_100minf"

# vocab = torch.load(f"saves/vocab_{version}.pt")
len(vocab)

6908

In [119]:
# torch.save(vocab, "saves/vocab_may1_WT2_transformer_min25f.pt")

In [5]:
from datasets import load_dataset
wikitext2 = load_dataset("wikitext", "wikitext-2-v1")

DATA_SPLIT = "train"
text = wikitext2[DATA_SPLIT]['text']
text = [item.lower().strip() for item in text if len(item) > 0]
text = [item.split(" ") + ["\n"] for item in text if "=" not in item]
# Prepare Corpus
DATA_LIMIT = None #paragraph limit
all_words = []
for paragraph in text[:DATA_LIMIT]:
    all_words += paragraph
all_words = pd.Series(all_words)
len(all_words)
len(all_words.unique())

v_counts = all_words.value_counts()

#filter out rare words
N_times = 25 #shows up in 100 different paragraphs
# v_counts = all_words.value_counts()
corpus = pd.Series([key for key in v_counts.keys() if v_counts[key] >= N_times])
corpus
vocab = {}
for i in range(len(corpus)):
    vocab[corpus[i]] = i
len(vocab)
#6908

6908

In [4]:
vocab = torch.load("saves/vocab_may1_WT2_transformer_min25f.pt")

In [5]:
def lookup_id(word, vocab=vocab):
    if word not in vocab:
        return vocab["<unk>"]
    return vocab[word]
def lookup_token(word_id, vocab=vocab):
    for word in vocab:
        if vocab[word] == word_id:
            return word
    return "<unk>"

In [7]:
def split_list(l, item, include_item = False):
    particular_value = item
    result = []
    temp_list = []
    for i in l:
        if i == particular_value:
            if include_item:
                temp_list.append(i)
            result.append(temp_list)
            temp_list = []
        else:
            temp_list.append(i)
    result.append(temp_list)
    return result
split_list(["5", "6", "7", "8", "9", "8", "9", "a", "b"], "c")

[['5', '6', '7', '8', '9', '8', '9', 'a', 'b']]

In [6]:
from datasets import load_dataset
wikitext2 = load_dataset("wikitext", "wikitext-103-v1")
text_train = wikitext2["train"]['text']
text_train = [item.lower().strip() for item in text_train if len(item) > 0]
text_test = wikitext2["test"]['text']
text_test = [item.lower().strip() for item in text_test if len(item) > 0]
# len(text_test)
text_train = [item.split(" ") + ["\n"] for item in text_train if "=" not in item]
text_test = [item.split(" ") + ["\n"] for item in text_test if "=" not in item]

In [7]:
min_seq_length = 32
max_seq_length = 128

buffer = 32

# x_train = [[lookup_id(word) for word in split_list(paragraph, ".", True)[1]] for paragraph in text_train if len(split_list(paragraph, ".", True))>=3]
# y_train = [item[1:] for item in x_train]
# x_train = [item[:-1] for item in x_train]

# x_test = [[lookup_id(word) for word in split_list(paragraph, ".", True)[1]] for paragraph in text_test if len(split_list(paragraph, ".", True))>=3]
# print(len(x_test))
# y_test = [item[1:] for item in x_test]
# x_test = [item[:-1] for item in x_test]
def seq_length():
    return random.randint(min_seq_length, max_seq_length)

In [8]:
x_train = []
y_train = []
batch_size = 40
batch_n = 0
batch_seq_length = seq_length()
count = 0
unk_id = lookup_id("<unk>")
unk_f_cutoff = 0.1
next_batch = False
for paragraph in text_train:
    if next_batch and len(x_train) % batch_size == 0:
        batch_n += 1
        batch_seq_length = seq_length()
        next_batch = False
    index = min(buffer, (paragraph.index(".")+1) if '.' in paragraph else len(paragraph))
    while index + batch_seq_length + 1 <= len(paragraph) and (not next_batch or len(x_train) % batch_size != 0):
        portion = paragraph[index:index + batch_seq_length+1]
        if sum([i not in vocab for i in portion])/len(portion) > unk_f_cutoff:
            # print("continuing", sum(np.array(portion)=="<unk>"), len(portion), portion)
            index += batch_seq_length
            continue
        x_train.append([lookup_id(word) for word in portion[:-1]])
        y_train.append([lookup_id(word) for word in portion[1:]])
        index += batch_seq_length
        next_batch = True
    

# x_train = [[lookup_id(word) for word in paragraph[buffer:max_seq_length+buffer+1]]  for paragraph in text_train if len(paragraph) >= max_seq_length + buffer+1]
# y_train = [item[1:] for item in x_train]
# x_train = [item[:-1] for item in x_train]

x_train = x_train[:-1]
y_train = y_train[:-1]
len(x_train)

197943

In [9]:
test_max_seq_length = 128
def test_seq_length():
    return random.randint(min_seq_length, test_max_seq_length)
x_test = [[lookup_id(word) for word in paragraph[buffer:test_seq_length()+buffer+1]] for paragraph in text_test if len(paragraph) >= test_max_seq_length + buffer+1]
y_test = [item[1:] for item in x_test]
x_test = [item[:-1] for item in x_test]
len(x_test[0]), len(x_test), len(x_train)/40

(32, 504, 4948.575)

In [10]:
unk_id = lookup_id("<unk>")
unk_prop = np.array([sum(np.array(item)==unk_id)/len(item) for item in x_train])
sum(unk_prop <= 0.1)

184849

In [11]:
print([lookup_token(i) for i in x_train[0]])
print([lookup_token(i) for i in y_train[0]])

['valkyria', 'of', 'the', 'battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', ',', 'is', 'a', 'tactical', 'role', '@-@', 'playing', 'video', 'game', 'developed', 'by', '<unk>', 'and', '<unk>', 'for', 'the', 'playstation', 'portable', '.', 'released', 'in', 'january', '2011', 'in', 'japan', ',', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series', '.', '<unk>', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@-@', 'time', 'gameplay', 'as', 'its', '<unk>', ',', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', '"', '<unk>', '"', ',', 'a', '<unk>', 'military', 'unit', 'serving', 'the', 'nation', 'of', '<unk>', 'during', 'the', 'second', '<unk>', 'war', 'who', 'perform', 'secret', 'black', 'operations', 'and', 'are', '<unk>', 'against', 'the', 'imperial', 'unit', '"', '<unk>', '<unk>', '"']
['of', 'the', 'battlefield', '3', ')', ',', 'commonly', 'referred

In [34]:
# x_train = x_train[6872:]
# y_train = y_train[6872:]
# x_test = x_test
# y_test = y_test


In [68]:
len(x_test[2])

71

In [69]:
len(x_train[0])

102

In [12]:
from transformer import TransformerModel

In [30]:
v_size = len(vocab)
d_model = 128
num_heads = 8
num_layers = 6
d_feedfoward = 2048
dropout = 0.5

transformer = TransformerModel(ntoken = v_size, ninp = d_model, nhead = num_heads, nhid = d_feedfoward, nlayers = num_layers, dropout = dropout)


/Users/gregengel/Documents/GitHub/Simple-Chatbot/.venv/lib/python3.9/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [31]:
import pytorch_warmup as warmup

In [32]:
net_file = "saves/model_transformer_may5_1130pm.pt"
# transformer = torch.load("saves/model_transformer_may4_0130pm.pt")

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9) #normal lr is 0.0001
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
warmup_period = 250
warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period = warmup_period)

transformer.train()

num_batches = len(x_train) // batch_size
BATCH_PRINT_SIZE = 100
percent_data_per_epoch = 0.9

print("Batches per epoch:", num_batches/BATCH_PRINT_SIZE * percent_data_per_epoch)



indices = list(range(num_batches))
for epoch in range(50):
    train_loss = 0
    # x_train_copy = [x_train[indices[i]] for i in range(len(indices))]
    # y_train_copy = [y_train[indices[i]] for i in range(len(indices))]
    x_train_copy = [x_train[indices[i]*batch_size:(indices[i]+1)*batch_size] for i in range(len(indices))]
    y_train_copy = [y_train[indices[i]*batch_size:(indices[i]+1)*batch_size] for i in range(len(indices))]
    for batch in range(int(num_batches * percent_data_per_epoch)):
        # source = torch.tensor(x_train_copy[batch*batch_size:(batch+1)*batch_size])  # (batch_size, seq_length)
        # target = torch.tensor(y_train_copy[batch*batch_size:(batch+1)*batch_size])  # (batch_size, seq_length)
        source = torch.tensor(x_train_copy[batch])  # (batch_size, seq_length)
        target = torch.tensor(y_train_copy[batch])  # (batch_size, seq_length)
        # print(source)
        # print(target)
        optimizer.zero_grad()
        output = transformer(source)
        output = output.view(-1, v_size)
        loss = criterion(output, target.view(-1))
        # print(output.shape)
        # print(target.view(-1).shape)
        # print("________")
        # print(loss)
        loss.backward()
        optimizer.step()
        if batch % BATCH_PRINT_SIZE == 0:
            print("|", end="")
        train_loss += loss.item()
    print(" -> ", end='')
    #eval
    test_loss = 0
    count_loss = 0
    with torch.no_grad():
        for datax, datay in zip(x_test,y_test):
            source = torch.tensor([datax])
            target = torch.tensor([datay]).view(-1)
            output = transformer(source)
            output = output.view(-1, v_size)
            test_loss += criterion(output, target).item() * len(datax)
            count_loss += len(datax)
    test_loss /= count_loss

    train_loss /= int(num_batches * percent_data_per_epoch)
    random.shuffle(indices)
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss}, Test Loss: {test_loss}")
    # warmup_scheduler.step()
    with warmup_scheduler.dampening():
        if warmup_scheduler.last_step + 1 >= warmup_period:
            lr_scheduler.step()
torch.save(transformer, net_file)

Batches per epoch: 44.532
||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 1, Train Loss: 8.313358076122565, Test Loss: 7.904851235771033
||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 2, Train Loss: 7.867592939440652, Test Loss: 7.457566364537351
||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 3, Train Loss: 7.392045249356437, Test Loss: 6.908466071583813
||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 4, Train Loss: 6.857980460872175, Test Loss: 6.391155748054967
||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 5, Train Loss: 6.4821508071568745, Test Loss: 6.126567196615718
||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 6, Train Loss: 6.317530802708113, Test Loss: 6.035394352376853
||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 7, Train Loss: 6.235146408479401, Test Loss: 5.975288279224146
||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 8, Train Loss: 6.167859269237883, Test Loss: 5.9150056886231335
||||||||||||||||||||

In [33]:
lr_scheduler.get_last_lr()

[0.0001]

In [162]:
o = transformer(src_data[0:1])
sm = np.array(torch.softmax(o, 1)[0].detach())
ids = [list(v).index(max(v)) for v in sm]
words = [lookup_token(i) for i in ids]
print([lookup_token(i) for i in src_data[0]])
print(words)

['card', '.', 'the', '<unk>', 'were', 'performed', 'on', 'the', 'fastra', 'ii', ',', 'the', 'fastra', 'i', ',', 'a', '<unk>', '@-@', 'core', '<unk>', '(', 'consisting', 'of', '<unk>', '<unk>', ')', ',', 'an', '<unk>', '<unk>', '<unk>', '<unk>', 'card', 'on', 'an', '<unk>', 'core', '<unk>', '<unk>', '<unk>', ',', 'and', 'on', 'an', '<unk>', 'core', '<unk>', '<unk>', '<unk>', 'itself', '.', 'the', 'fastra', 'ii', 'is', 'over', 'three', 'times', 'faster', 'than', 'the', 'fastra', 'i', 'in', '<unk>', '<unk>', 'reconstruction', 'speed', '.', 'although', 'the', 'fastra', 'ii', '<unk>', 'more', 'power', 'than', 'the', 'fastra', 'i', ',', 'it', "'s", 'nearly', '3', 'times', 'as', 'energy', '<unk>', 'as', 'the', 'fastra', 'i', ',', 'and', 'over', '300', 'times', 'as', 'energy', '<unk>', 'as', 'the', '<unk>', '@-@', 'core', '<unk>', '.', 'the', 'video', 'cards', 'run', 'at', '37', 'degrees', '<unk>', 'when', '<unk>', ',', 'and', 'at', '60', 'degrees', '<unk>', 'at', 'full', 'load', '.']
['with',

In [21]:
text = "you are a helpful assistant . Answer the following question . "

text = [lookup_id(word.lower()) for word in text.strip().split(" ")]

for count in range(50):
    i = torch.tensor([text])
    o = transformer(i)
    sm = np.array(torch.softmax(o, 2)[0].detach())
    top5 = [np.zeros(5) for _ in range(len(sm))]
    top5p = [np.zeros(5) for _ in range(len(sm))]
    for vi in range(len(sm)-1,len(sm)):
        v = sm[vi]
        for item in v:
            m = top5[vi][list(top5[vi]).index(min(top5[vi]))]
            if lookup_token(list(v).index(item)) != "<unk>":
                top5[vi][list(top5[vi]).index(min(top5[vi]))] = max(m, item)
        top5[vi] = [list(v).index(i) for i in top5[vi]]
        top5p[vi] = [v[i] for i in top5[vi]]
    # ids = [list(v).index(max(v)) for v in sm]
    words = [[lookup_token(i) for i in w] for w in top5][-1]
    # print([lookup_token(i) for i in src_data[0]])
    # print(words)
    print(' '.join([lookup_token(i) for i in text]))
    print(words)
    print(top5p[-1])
    chosen_word = words[int(input("Choose a word: "))-1]
    text.append(lookup_id(chosen_word))
# text


you are a <unk> assistant . answer the following question .
['in', 'it', '"', 'he', 'on']
[0.062019486, 0.050097946, 0.06209897, 0.055168714, 0.036025554]
you are a <unk> assistant . answer the following question . on
['a', '1', '3', 'august', '8']
[0.028086705, 0.041982103, 0.016172329, 0.01959111, 0.016415324]
you are a <unk> assistant . answer the following question . on august
['6', '5', '2', '1', '8']
[0.026780343, 0.029847339, 0.02555999, 0.032186054, 0.025736617]
you are a <unk> assistant . answer the following question . on august 6
['@.@', ',', '.', '@-@', '@,@']
[0.06803521, 0.07280752, 0.028437186, 0.037841234, 0.05639462]
you are a <unk> assistant . answer the following question . on august 6 ,
['but', 'which', '"', 'he', 'and']
[0.03431506, 0.028054025, 0.026013533, 0.033768643, 0.13182275]
you are a <unk> assistant . answer the following question . on august 6 , which
['were', ',', '.', 'was', 'had']
[0.060836777, 0.035716124, 0.025741752, 0.056733724, 0.02901606]
you are

ValueError: invalid literal for int() with base 10: '?'

In [14]:
device = torch.device("cpu")
ntokens = len(vocab)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)
model = transformer
temperature = 0.5
log_interval = 1
input = 'i am a big baseball fan . i love the sights and the '
input = [lookup_id(i) for i in input.strip().split(" ")]
input = torch.tensor(input).view(len(input), 1)


with open('out_generation.txt', 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(20):
            output = model(input, False)
            word_weights = output[-1].squeeze().div(temperature).exp().cpu()
            word_weights[lookup_id("<unk>")] = 0
            word_idx = torch.multinomial(word_weights, 1)[0]
            word_tensor = torch.Tensor([[word_idx]]).long().to(device)
            print(word_tensor)
            input = torch.cat([input, word_tensor], 0)

            word = lookup_token(word_idx)

            outf.write(word + ('\n' if i % 20 == 19 else ' '))

            if i % log_interval == 0:
                print('| Generated {}/{} words'.format(i, [lookup_token(i[0])for i in input]))

tensor([[267]])
| Generated 0/['i', 'am', 'a', 'big', 'baseball', 'fan', '.', 'i', 'love', 'the', '<unk>', 'and', 'the', 'last'] words
tensor([[49]])
| Generated 1/['i', 'am', 'a', 'big', 'baseball', 'fan', '.', 'i', 'love', 'the', '<unk>', 'and', 'the', 'last', 'two'] words
tensor([[2173]])
| Generated 2/['i', 'am', 'a', 'big', 'baseball', 'fan', '.', 'i', 'love', 'the', '<unk>', 'and', 'the', 'last', 'two', 'decades'] words
tensor([[2]])
| Generated 3/['i', 'am', 'a', 'big', 'baseball', 'fan', '.', 'i', 'love', 'the', '<unk>', 'and', 'the', 'last', 'two', 'decades', '.'] words
tensor([[9]])
| Generated 4/['i', 'am', 'a', 'big', 'baseball', 'fan', '.', 'i', 'love', 'the', '<unk>', 'and', 'the', 'last', 'two', 'decades', '.', '"'] words
tensor([[1]])
| Generated 5/['i', 'am', 'a', 'big', 'baseball', 'fan', '.', 'i', 'love', 'the', '<unk>', 'and', 'the', 'last', 'two', 'decades', '.', '"', ','] words
tensor([[5]])
| Generated 6/['i', 'am', 'a', 'big', 'baseball', 'fan', '.', 'i', 'love'